In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
import os
import cv2
import pathlib
import glob
import re
import sklearn

from tensorflow import keras
from tensorflow.keras import layers, datasets , models
from tensorflow.keras.models import Sequential

from PIL import Image

from sklearn.model_selection import train_test_split

In [11]:
image_labels = {
    'angry' : 0,
    'disgust' : 1,
    'fear' : 2,
    'happy' : 3,
    'neutral' : 4,
    'sad' : 5,
    'surprise' : 6
}

In [12]:
train_data = {}

for i in image_labels:
    train_data[i] = glob.glob('../input/face-dataset/images/images/train/' + i + '/*.jpg')

In [13]:
train_data['surprise'][:5]

['../input/face-dataset/images/images/train/surprise/30778.jpg',
 '../input/face-dataset/images/images/train/surprise/22706.jpg',
 '../input/face-dataset/images/images/train/surprise/20513.jpg',
 '../input/face-dataset/images/images/train/surprise/27582.jpg',
 '../input/face-dataset/images/images/train/surprise/33678.jpg']

In [14]:
test_data = {}

for i in image_labels:
    test_data[i] = glob.glob('../input/face-dataset/images/images/validation/' + i + '/*.jpg')

In [15]:
test_data['surprise'][:5]

['../input/face-dataset/images/images/validation/surprise/13288.jpg',
 '../input/face-dataset/images/images/validation/surprise/24201.jpg',
 '../input/face-dataset/images/images/validation/surprise/26556.jpg',
 '../input/face-dataset/images/images/validation/surprise/26076.jpg',
 '../input/face-dataset/images/images/validation/surprise/27577.jpg']

In [16]:
image_size = 48

x_train = []
y_train = []

for label,images in train_data.items():
    for i in images:
        x_train.append(np.array(Image.open(i)).reshape(image_size,image_size,-1))
        y_train.append(image_labels[label])

In [17]:
x_test = []
y_test = []

for label,images in test_data.items():
    for i in images:
        x_test.append(np.array(Image.open(i)).reshape(image_size,image_size,-1))
        y_test.append(image_labels[label])

In [18]:
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [19]:
x_train = x_train/255
x_test = x_test/255

In [20]:
x_train.shape

(28821, 48, 48, 1)

In [21]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(48,48,1)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

2021-11-20 14:00:31.079533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 14:00:31.194770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 14:00:31.195842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 14:00:31.197560: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [22]:
    model = Sequential([
        data_augmentation,
    #     cnn
      layers.Conv2D(64,(3,3), activation='relu',padding = 'same', input_shape = (48,48,1)),
      layers.MaxPooling2D(pool_size = (2,2)),

      layers.Conv2D(128,(3,3), activation='relu',padding = 'same'),
      layers.MaxPooling2D(pool_size = (2,2)),

      layers.Conv2D(128,(3,3), activation='relu',padding = 'same'),
      layers.MaxPooling2D(pool_size = (2,2)),

#       layers.Conv2D(512,(3,3), activation='relu',padding = 'same'),
#       layers.MaxPooling2D(pool_size = (2,2)),

    #     dense
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dropout(0.3),
#       layers.Dense(512, activation='relu'),
#       layers.Dropout(0.5),
      layers.Dense(7,activation='softmax')
    ])


    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
              
model.fit(x_train,y_train, epochs=150) 

2021-11-20 14:00:34.847242: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150


/opt/conda/lib/python3.7/site-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '
2021-11-20 14:00:37.113984: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


901/901 [==============================] - 15s 8ms/step - loss: 1.7770 - accuracy: 0.2598
Epoch 2/150
901/901 [==============================] - 7s 7ms/step - loss: 1.6454 - accuracy: 0.3481
Epoch 3/150
901/901 [==============================] - 7s 8ms/step - loss: 1.5228 - accuracy: 0.4107
Epoch 4/150
901/901 [==============================] - 7s 8ms/step - loss: 1.4555 - accuracy: 0.4371
Epoch 5/150
901/901 [==============================] - 7s 7ms/step - loss: 1.3992 - accuracy: 0.4602
Epoch 6/150
901/901 [==============================] - 8s 8ms/step - loss: 1.3629 - accuracy: 0.4765
Epoch 7/150
901/901 [==============================] - 7s 8ms/step - loss: 1.3353 - accuracy: 0.4896
Epoch 8/150
901/901 [==============================] - 7s 7ms/step - loss: 1.3108 - accuracy: 0.5007
Epoch 9/150
901/901 [==============================] - 7s 8ms/step - loss: 1.2921 - accuracy: 0.5050
Epoch 10/150
901/901 [==============================] - 7s 7ms/step - loss: 1.2776 - accuracy: 0.5100


In [23]:
model.fit(x_train,y_train, epochs=50) 

Epoch 1/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9552 - accuracy: 0.6355
Epoch 2/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9472 - accuracy: 0.6383
Epoch 3/50
901/901 [==============================] - 6s 7ms/step - loss: 0.9515 - accuracy: 0.6392
Epoch 4/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9466 - accuracy: 0.6406
Epoch 5/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9443 - accuracy: 0.6414
Epoch 6/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9449 - accuracy: 0.6389
Epoch 7/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9508 - accuracy: 0.6390
Epoch 8/50
901/901 [==============================] - 7s 7ms/step - loss: 0.9462 - accuracy: 0.6378
Epoch 9/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9432 - accuracy: 0.6434
Epoch 10/50
901/901 [==============================] - 7s 8ms/step - loss: 0.9517 - accuracy: 0.6398

In [24]:
model.evaluate(x_test,y_test)

221/221 [==============================] - 1s 3ms/step - loss: 1.1866 - accuracy: 0.5952


[1.1865519285202026, 0.595244824886322]

In [1]:
model.save('expression_detection_model.h5')

NameError: name 'model' is not defined

In [28]:
# # save model structure in jason file
# model_json = model.to_json()
# with open("emotion_model.json", "w") as json_file:
#     json_file.write(model_json)

# # save trained model weight in .h5 file
# model.save_weights('emotion_model.h5')

In [26]:
# import tensorflow_hub as hub

# feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

# pretrained_model_without_top_layer = hub.KerasLayer(
#     feature_extractor_model, input_shape=(48,48, 1), trainable=False)

In [27]:
# num_of_expressions = 7

# model = tf.keras.Sequential([
#   pretrained_model_without_top_layer,
#   tf.keras.layers.Dense(7)
# ])

# model.summary()